<a href="https://www.kaggle.com/code/manjitbaishya001/iris-binary-classification-ann?scriptVersionId=163681065" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

import os

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

In [4]:
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [5]:
torch.cuda.memory_reserved()

0

In [6]:
x = torch.FloatTensor([1.0, 2.0, 3.0])

In [7]:
x.device

device(type='cpu')

In [8]:
y = torch.FloatTensor([1.3, 4.5, 7.7]).cuda()

In [9]:
y.device

device(type='cuda', index=0)

In [10]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        src = os.path.join(dirname, filename)

In [11]:
df = pd.read_csv(src)

In [12]:
df["species"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [13]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [14]:
species_map = {'Iris-setosa' : 0.0, 'Iris-versicolor' : 1.0, 'Iris-virginica': 2.0}
df["species"] = df["species"].map(species_map)

In [15]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [16]:
X = df.drop("species", axis = 1)
y = df["species"]

In [17]:
# converting to numpy array
X = X.values
y = y.values

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 41)

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [20]:
# converting features and targest to tensors
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()

y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [21]:
X_train.device

device(type='cuda', index=0)

In [22]:
# creating a Model Class that inherits nn.Module
class ANN_Model(nn.Module):
    # input_layer -> 4 features
    # hidden_layer1(h1) -> h2 -> h3
    # output (3 classes of irirs flowers)
    
    def __init__(self, in_features = 4, h1=20, h2=20, h3=50, out_features = 3):
        super().__init__() #instantiate our nn.Module
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, h3)
        self.out = nn.Linear(h3, out_features)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.out(x)
        return x

In [23]:
torch.manual_seed(20)
model = ANN_Model()

In [24]:
model.parameters

<bound method Module.parameters of ANN_Model(
  (fc1): Linear(in_features=4, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=50, bias=True)
  (out): Linear(in_features=50, out_features=3, bias=True)
)>

In [25]:
for i in model.parameters():
    print(i.is_cuda)

False
False
False
False
False
False
False
False


In [26]:
model = model.cuda()

In [27]:
for i in model.parameters():
    print(i.is_cuda)

True
True
True
True
True
True
True
True


In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [29]:
# determining epochs
epochs = 100

# tracking our losses
losses = []

for i in range(epochs):
    y_pred = model.forward(X_train)
    
    # measure loss
    loss = criterion(y_pred, y_train)
    
    # keeping track of losses
    losses.append(loss.detach())
    
    # print every 10 epochs
    if i%10 == 9:
        print(f'Epoch: {i} and loss: {loss}')
    
    # back propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 9 and loss: 0.7017250061035156
Epoch: 19 and loss: 0.3402673006057739
Epoch: 29 and loss: 0.131489098072052
Epoch: 39 and loss: 0.06595204025506973
Epoch: 49 and loss: 0.04647845774888992
Epoch: 59 and loss: 0.0348275750875473
Epoch: 69 and loss: 0.028868593275547028
Epoch: 79 and loss: 0.026037510484457016
Epoch: 89 and loss: 0.02381807379424572
Epoch: 99 and loss: 0.02549036592245102
